In [16]:
#2018 HMDA Edit Testing File Generator
from collections import OrderedDict
from io import StringIO
import json
import os
import pandas as pd
import random

#custom imports
import lar_constraints
import lar_generator
from rules_engine import rules_engine


In [17]:
#2018 Filing Instruction Guide: https://www.consumerfinance.gov/data-research/hmda/static/for-filers/2018/2018-HMDA-FIG.pdf

use_cols = ['name', 'metDivName', 'countyFips', 'geoIdMsa', 'metDivFp', 'smallCounty', 'tracts']
cbsa_cols = ['name', 'metDivName', 'state', 'countyFips', 'county', 'tracts','geoIdMsa', 'metDivFp', 'smallCounty', 
             'stateCode', 'tractDecimal']
cbsas = pd.read_csv('../dependancies/tract_to_cbsa_2015.txt', usecols=use_cols, delimiter='|', 
                    header=None, names=cbsa_cols, dtype=str) #load tract to CBSA data from platform file
cbsas["tractFips"] = cbsas.countyFips + cbsas.tracts
counties = list(cbsas.countyFips)
tracts = list(cbsas.tractFips)

In [18]:
#load schemas for LAR and transmittal sheet
lar_schema_df = pd.DataFrame(json.load(open("../schemas/lar_schema.json", "r")))
ts_schema_df = pd.DataFrame(json.load(open("../schemas/ts_schema.json", "r")))

In [19]:
lar_gen = lar_generator.lar_gen(lar_schema_df, ts_schema_df, counties=counties, tracts=tracts) #instantiate generator
lar_const = lar_constraints.lar_constraints(counties=counties, tracts=tracts)#instantiate constraints


In [20]:
#run constraint functions on row
file_length = 5 #set number of rows in test file
first = True
lei = None

for i in range(0, file_length):
    print("making new row\n\n\n\n\n\n\n")
    if lei:
        row = lar_gen.make_row(lei=lei)
    else:
        row = lar_gen.make_row() #create new row
    lei = row["lei"]
    diff = [1] #initialize diff for loop
    iters = 0
    while len(diff) > 0:
        row_base = row.copy() #copy row to enable diff
        #apply constraint functions to LAR row
        for func in lar_const.constraint_funcs: 
            print("starting constraints\n\n\n\n\n")
            row = getattr(lar_const, func)(row)
            #convert initial and copied rows to sets for diff
            initial_row = set(row_base.items())
            changed_row = set(row.items())
            diff = (initial_row - changed_row) #subtract row sets to show changes from constraint funcs
#            print(len(initial_row), len(changed_row)) #check number of fields, should be 110
            #show readout of number of changes made to new LAR row
            print("changes from {func}: {vals}".format(func=str(func), vals=len(diff)))
            #print(row["app_date"], row["action_taken"])
            print(diff)
        iters+=1
    #create first row of dataframe
    if first:
        lar_frame = pd.DataFrame(row, index=[1])
        first = False
    #add additional rows to dataframe
    else:
        #print("concating")
        new_lar = pd.DataFrame(row, index=[1])
        lar_frame = pd.concat([lar_frame, new_lar], axis=0)
lar_frame.reset_index(inplace=True) #reset index
lar_frame.drop("index", inplace=True, axis=1) #drop additional index column
print(iters)

making new row







starting constraints





changes from v612_const: 1
{('loan_purpose', '2')}
starting constraints





changes from v610_const: 1
{('loan_purpose', '2')}
starting constraints





changes from v613_const: 1
{('loan_purpose', '2')}
starting constraints





changes from v614_const: 2
{('preapproval', '1'), ('loan_purpose', '2')}
starting constraints





changes from v615_const: 2
{('preapproval', '1'), ('loan_purpose', '2')}
starting constraints





changes from v619_const: 2
{('preapproval', '1'), ('loan_purpose', '2')}
starting constraints





changes from v622_const: 2
{('preapproval', '1'), ('loan_purpose', '2')}
starting constraints





changes from v627_const: 2
{('preapproval', '1'), ('loan_purpose', '2')}
starting constraints





changes from v628_const: 6
{('loan_purpose', '2'), ('app_eth_5', '12'), ('app_eth_3', '1'), ('app_eth_4', '14'), ('preapproval', '1'), ('app_eth_2', '14')}
starting constraints





changes from v629_const: 7
{('loan_purpose',

changes from v696_const: 5
{('denial_2', ''), ('app_race_4', '5'), ('app_eth_1', '2'), ('denial_3', ''), ('denial_4', '')}
starting constraints





changes from v697_const: 5
{('denial_2', ''), ('app_race_4', '5'), ('app_eth_1', '2'), ('denial_3', ''), ('denial_4', '')}
starting constraints





changes from v698_const: 5
{('denial_2', ''), ('app_race_4', '5'), ('app_eth_1', '2'), ('denial_3', ''), ('denial_4', '')}
starting constraints





changes from v699_const: 5
{('denial_2', ''), ('app_race_4', '5'), ('app_eth_1', '2'), ('denial_3', ''), ('denial_4', '')}
starting constraints





changes from v700_const: 5
{('denial_2', ''), ('app_race_4', '5'), ('app_eth_1', '2'), ('denial_3', ''), ('denial_4', '')}
starting constraints





changes from v701_const: 5
{('denial_2', ''), ('app_race_4', '5'), ('app_eth_1', '2'), ('denial_3', ''), ('denial_4', '')}
starting constraints





changes from v702_const: 5
{('denial_2', ''), ('app_race_4', '5'), ('app_eth_1', '2'), ('denial_3', ''), (

{('denial_2', '4'), ('denial_3', '3')}
starting constraints





changes from v699_const: 2
{('denial_2', '4'), ('denial_3', '3')}
starting constraints





changes from v700_const: 2
{('denial_2', '4'), ('denial_3', '3')}
starting constraints





changes from v701_const: 2
{('denial_2', '4'), ('denial_3', '3')}
starting constraints





changes from v702_const: 2
{('denial_2', '4'), ('denial_3', '3')}
starting constraints





changes from v703_const: 2
{('denial_2', '4'), ('denial_3', '3')}
starting constraints





changes from v704_const: 2
{('denial_2', '4'), ('denial_3', '3')}
starting constraints





changes from v705_const: 2
{('denial_2', '4'), ('denial_3', '3')}
starting constraints





changes from v612_const: 0
set()
starting constraints





changes from v610_const: 0
set()
starting constraints





changes from v613_const: 0
set()
starting constraints





changes from v614_const: 0
set()
starting constraints





changes from v615_const: 0
set()
starting constraints



changes from v662_const: 5
{('co_app_eth_3', '14'), ('co_app_eth_5', '1'), ('co_app_eth_2', '12'), ('co_app_eth_4', '11'), ('app_date', '20181103')}
starting constraints





changes from v663_const: 7
{('app_credit_score', '444'), ('co_app_eth_3', '14'), ('co_app_eth_5', '1'), ('co_app_eth_2', '12'), ('co_app_eth_4', '11'), ('app_score_name', '1'), ('app_date', '20181103')}
starting constraints





changes from v664_const: 9
{('co_app_credit_score', '890'), ('app_credit_score', '444'), ('co_app_score_name', '1'), ('co_app_eth_3', '14'), ('co_app_eth_5', '1'), ('co_app_eth_2', '12'), ('co_app_eth_4', '11'), ('app_score_name', '1'), ('app_date', '20181103')}
starting constraints





changes from v666_const: 9
{('co_app_credit_score', '890'), ('app_credit_score', '444'), ('co_app_score_name', '1'), ('co_app_eth_3', '14'), ('co_app_eth_5', '1'), ('co_app_eth_2', '12'), ('co_app_eth_4', '11'), ('app_score_name', '1'), ('app_date', '20181103')}
starting constraints





changes from v667_

In [21]:
#Quality and Macro field interrelationship constraints:
print(iters)

4


In [22]:
#Create a sample TS row
#Note: this will need to be more robust to include other federal agencies
# mlo_id needs NA option
#set dummy values for TS row
ts_row_small = OrderedDict()
ts_row_small["record_id"]="1"
ts_row_small["inst_name"]="Ficus Bank"
ts_row_small["calendar_year"]=str(2018)
ts_row_small["calendar_quarter"]="4"
ts_row_small["contact_name"]="Mr. Smug Pockets"
ts_row_small["contact_tel"]="555-555-5555"
ts_row_small["contact_email"]="pockets@ficus.com"
ts_row_small["contact_street_address"]="1234 Ficus Lane"
ts_row_small["office_city"]="Ficusville"
ts_row_small["office_state"]="UT"
ts_row_small["office_zip"]="84096"
ts_row_small["federal_agency"]="9"
ts_row_small["lar_entries"]= str(len(lar_frame))
ts_row_small["tax_id"]="01-0123456"
ts_row_small["lei"]=lar_frame.get_value(0, "lei")

In [23]:
#join LAR and TS rows to make an output file
def write_file(ts_input=None, lar_input=None, directory="../edits_files/", name="passes_all.txt"):
    """Takes a TS row as a dictionary and LAR data as a dataframe. Writes LAR data to file and 
    re-reads it to combine with TS data to make a full file."""
    #make directories for files if they do not exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    #write LAR dataframe to file
    parts_dir = directory+"file_parts/"
    if not os.path.exists(parts_dir):
        os.makedirs(parts_dir)
        
    lar_input.to_csv(parts_dir + "lar_data.txt", sep="|", header=False, index=False, index_label=False)
    #load LAR data as file rows
    with open(parts_dir + "lar_data.txt", 'r') as lar_data:
        lar = lar_data.readlines()

    with open(directory + name, 'w') as final_file:
        final_file.write("|".join(ts_input.values())+"\n")
        for line in lar:
            final_file.write("{line}".format(line=line))

In [24]:
#modify file for testing
lar_fields = []
ts_fields = []
for field in lar_fields:
    lar_frame[field] = "1"

for field in ts_fields:
    ts_row_small[field] = ""

print(ts_row_small)

OrderedDict([('record_id', '1'), ('inst_name', 'Ficus Bank'), ('calendar_year', '2018'), ('calendar_quarter', '4'), ('contact_name', 'Mr. Smug Pockets'), ('contact_tel', '555-555-5555'), ('contact_email', 'pockets@ficus.com'), ('contact_street_address', '1234 Ficus Lane'), ('office_city', 'Ficusville'), ('office_state', 'UT'), ('office_zip', '84096'), ('federal_agency', '9'), ('lar_entries', '5'), ('tax_id', '01-0123456'), ('lei', 'YEIG9VQAHAAKF0GAKSUI')])


In [25]:
#write sample file to disk
write_file(ts_input=ts_row_small, lar_input=lar_frame) #writes created file to disk
#validator engine uses the default: path="../edits_files/", data_file="passes_all.txt" for data files
validator = rules_engine(lar_schema_df, ts_schema_df) #instantiate edits rules engine

In [26]:
#split TS and LAR using validator function
#validator creates class objects of each of these internally as well
ts_df, lar_df = validator.split_ts_row(path="../edits_files/", data_file="passes_all.txt")

In [27]:
for func in dir(validator):
    if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
        print("applying:", func)
        getattr(validator, func)()


applying: s300_1
applying: s300_2
applying: s301
applying: s302
applying: s304
applying: s305
applying: v600
applying: v601_1
applying: v601_2
applying: v601_3
applying: v601_4
applying: v601_5
applying: v602
applying: v603
applying: v604
applying: v605
applying: v606
applying: v607
applying: v608
applying: v609
applying: v610_1
applying: v610_2
applying: v611
applying: v612_1
applying: v612_2
applying: v613_1
applying: v613_2
applying: v613_3
applying: v613_4


In [28]:
validator.results

{'s300_1': OrderedDict([('row_type', 'TS'), ('record_id', 'passed')]),
 's300_2': OrderedDict([('row_type', 'LAR'), ('record_id', 'passed')]),
 's301': OrderedDict([('row_type', 'TS'), ('LEI', 'passed')]),
 's302': OrderedDict([('row_type', 'TS'), ('calendar_year', 'passed')]),
 's304': OrderedDict([('row_type', 'TS/LAR'), ('lar_entries', 'passed')]),
 's305': OrderedDict([('row_type', 'LAR'), ('all', 'passed')]),
 'v600': OrderedDict([('row_type', 'LAR'), ('LEI', 'passed')]),
 'v601_1': OrderedDict([('row_type', 'TS'), ('inst_name', 'passed')]),
 'v601_2': OrderedDict([('row_type', 'TS'), ('contact_name', 'passed')]),
 'v601_3': OrderedDict([('row_type', 'TS'), ('contact_email', 'passed')]),
 'v601_4': OrderedDict([('row_type', 'TS'),
              ('contact_street_address', 'passed')]),
 'v601_5': OrderedDict([('row_type', 'TS'), ('office_city', 'passed')]),
 'v602': OrderedDict([('row_type', 'TS'), ('calendar_quarter', 'passed')]),
 'v603': OrderedDict([('row_type', 'TS'), ('contact

In [29]:
validator.lar_df[["action_taken", "preapproval"]]

,action_taken,preapproval
0,3,2
1,5,2
2,3,2
3,6,2
4,3,2


In [30]:
validator.lar_df[(lar_df.preapproval=="1")]#&((lar_df.action_taken.isin(("1","2","7","8"))))]

,record_id,lei,uli,app_date,loan_type,loan_purpose,preapproval,const_method,occ_type,loan_amount,...,aus_code_5,aus_result_1,aus_result_2,aus_result_3,aus_result_4,aus_result_5,aus_code_16,reverse_mortgage,open_end_credit,business_purpose
